# Bayesian analysis and MCMC

## Defining priors

In [ ]:
from likelihood import JointPriors

# Defining the priors and the type of distribuition
param_configs_frb = {
    'H0': ((20, 100), 'uniform'), 
    #'Omega_bh2': ((0.02186, 0.02284), 'uniform'),
    'Omega_bh2': ((0.02235, 0.00049), 'gaussian'),
    'q0': ((-0.95, -0.3), 'uniform'),
    'j0': ((0, 2), 'uniform'),
    # 's0': ((-10, -5), 'uniform'),
    'A': ((55, 225), 'uniform'),
    'beta': ((0, 4), 'uniform')
}

param_configs_sne = {
    'H0': ((20, 100), 'uniform'), 
    'q0': ((-0.95, -0.3), 'uniform'),
    'j0': ((0, 2), 'uniform'),
    # 's0': ((-10, -5), 'uniform'),
    #'Mb': ((-19.245, 0.029), 'gaussian')
}

param_configs_cc = {
    'H0': ((20, 100), 'uniform'), 
    'q0': ((-0.95, -0.3), 'uniform'),
    'j0': ((0, 2), 'uniform'),
    # 's0': ((-10, -5), 'uniform')
}

# Definir apenas os parâmetros compartilhados para a análise conjunta
shared_param_configs = {
    'H0': ((20, 100), 'uniform'), 
    'q0': ((-0.95, -0.3), 'uniform'),
    'j0': ((0, 2), 'uniform')
}

P_frb = JointPriors(param_configs_frb)
P_sne = JointPriors(param_configs_sne)
P_cc = JointPriors(param_configs_cc)
P_shared = JointPriors(shared_param_configs)

### Models and parameters

In [ ]:
from likelihood import JointLikelihoodFunction
from equations import Cosmography

model = Cosmography()

# Creating an instance of JointLikelihoodFunction
LF_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, H0, Omega_bh2, q0, j0, A, beta: model.DM_ext_th_cmy(
        z=z,
        f_IGM=0.83,
        Omega_bh2=Omega_bh2, # from Planck 2018 0.0224 
        H0=H0,
        q0=q0,
        j0=j0,
        # s0=s0,
        A=A,
        beta=beta
    ) 
    }
)

LF_sne = JointLikelihoodFunction(
    { 'SNe': lambda z, H0, q0, j0: model.Mu_cmy(
        z=z,
        H0=H0,
        q0=q0,
        j0=j0,
        # s0=s0
    ) 
    }
)

LF_cc = JointLikelihoodFunction(
    { 'CC': lambda z, H0, q0, j0: model.H_cmy(
        z=z, 
        H0=H0,
        q0=q0,
        j0=j0,
        # s0=s0
    ) 
    }
)

LF_frb_sne_cc = JointLikelihoodFunction(
    { 'FRB': lambda z, H0, Omega_bh2, q0, j0, A, beta: model.DM_ext_th_cmy(
        z=z,
        f_IGM=0.83,
        Omega_bh2=Omega_bh2, # from Planck 2018  
        H0=H0,
        q0=q0,
        j0=j0,
        # s0=s0,
        A=A,
        beta=beta
    ),

        'SNe': lambda z, H0, q0, j0: model.Mu_cmy(
        z=z,
        H0=H0,
        q0=q0,
        j0=j0,
        # s0=s0
    ),
        
        'CC': lambda z, H0, q0, j0: model.H_cmy(
        z=z, 
        H0=H0,
        q0=q0,
        j0=j0
    ) 
    },

    ["H0", "q0", "j0"]
)

## Preparing the samples

### Analysis for 66 FRBs

In [ ]:
from obs_data import FRB_data, SNe_data, H_data
import ultranest

# Instantiate the FRB_data class for 5000 FRB mock
frb_mock = FRB_data(n_frb=5000)

# Call the select_data method to get the observed data
z_frb, DM_ext, sigma_ext = frb_mock.select_data()

sne_data_pantheon = SNe_data(sample_sne='Pantheon+SH0ES')

z_sne, mu_sne, cov_matrix_inv = sne_data_pantheon.load_data()

# sne_data_des = SNe_data(sample_sne='DESY5')

# z_sne, mu_sne, cov_matrix_inv = sne_data_des.load_data()

Hz_data = H_data()

z_val, H_val, errors = Hz_data.H_z_data() 

# Configuring the ultranest samplers
sampler_frb = ultranest.ReactiveNestedSampler(
    P_frb.param_names,
    lambda params: LF_frb.log_likelihood(
        dict(zip(P_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext)
        }
    ),
    P_frb.prior_transform
)

sampler_sne = ultranest.ReactiveNestedSampler(
    P_sne.param_names,
    lambda params: LF_sne.log_likelihood(
        dict(zip(P_sne.param_names, params)),
        {
            'SNe': (z_sne, mu_sne, cov_matrix_inv)
        }
    ),
    P_sne.prior_transform
)

sampler_cc = ultranest.ReactiveNestedSampler(
    P_cc.param_names,
    lambda params: LF_cc.log_likelihood(
        dict(zip(P_cc.param_names, params)),
        {
            'CC': (z_val, H_val, errors)
        }
    ),
    P_cc.prior_transform
)

sampler_frb_sne_cc = ultranest.ReactiveNestedSampler(
    P_frb.param_names,
    lambda params: LF_frb_sne_cc.log_likelihood(
        dict(zip(P_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext),
            'CC': (z_val, H_val, errors),
            'SNe': (z_sne, mu_sne, cov_matrix_inv)
        }
    ),
    P_frb.prior_transform
)

In [ ]:
frb = sampler_frb.run(min_num_live_points=400)
sampler_frb.print_results()

In [ ]:
cc = sampler_cc.run(min_num_live_points=400)
sampler_cc.print_results()

In [ ]:
sne = sampler_sne.run(min_num_live_points=400)
sampler_sne.print_results()

In [ ]:
frb_sne_cc = sampler_frb_sne_cc.run(min_num_live_points=400)
sampler_frb_sne_cc.print_results()

In [ ]:
from getdist import plots, MCSamples
import numpy as np

# Extraindo amostras dos resultados para frb
samples_frb = frb['samples']
samples_cc = cc['samples']
samples_sne = sne['samples']
samples_frb_sne_cc = frb_sne_cc['samples']

labels_frb = ['H_0', '\\Omega_{b}h^2', 'q_0', 'j_0', 'A', '\\beta']
names_frb = P_frb.param_names

labels_cc = ['H_0', 'q_0', 'j_0']
names_cc = P_cc.param_names

"""# Calculando o parâmetro derivado
derived_param_gp = 100 * np.sqrt(samples_gp_frb[:, 0] / 0.0493)
derived_param_ann = 100 * np.sqrt(samples_ann_frb[:, 0] / 0.0493)

# Adicionando o parâmetro derivado nas amostras
samples_with_derived_gp = np.hstack([samples_gp_frb, derived_param_gp[:, None]])
samples_with_derived_ann = np.hstack([samples_ann_frb, derived_param_ann[:, None]])

names_extended = names1 + ['H0']
labels_extended = ['\\Omega_{b}h^2', 'A', '\\beta', 'H_0']"""

# Criando MCSamples com o parâmetro derivado
mcsamples_frb = MCSamples(samples=samples_frb, 
                           names=names_frb, 
                           labels=labels_frb)

mcsamples_cc = MCSamples(samples=samples_cc, 
                           names=names_cc, 
                           labels=labels_cc)

mcsamples_sne = MCSamples(samples=samples_sne, 
                           names=names_cc, 
                           labels=labels_cc)

mcsamples_frb_sne_cc = MCSamples(samples=samples_frb_sne_cc, 
                           names=names_frb, 
                           labels=labels_frb)

In [ ]:
mcsamples_frb.getMargeStats().saveAsText('Results_mcmc/marginalized_params_cosmography_frb_mock.txt')
# mcsamples_cc.getMargeStats().saveAsText('Results_mcmc/marginalized_params_cosmography_cc.txt')
# mcsamples_sne.getMargeStats().saveAsText('Results_mcmc/marginalized_params_cosmography_sne.txt')
mcsamples_frb_sne_cc.getMargeStats().saveAsText('Results_mcmc/marginalized_params_cosmography_frb_sne_cc_mock.txt')

In [ ]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_sne.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_cc.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_frb_sne_cc.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples_frb, mcsamples_cc, mcsamples_sne, mcsamples_frb_sne_cc], names_cc, filled=True, contour_colors=['green', 'red', 'blue', 'purple'], 
                legend_labels=['FRB', 'CC', 'SNe', 'FRB + CC + SNe'])
g.export('Figuras/joint_frb_sne_cc_mock.png', dpi=600)

In [ ]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples_cc.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_frb_sne_cc.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples_sne, mcsamples_frb_sne_cc], names_cc, filled=True, contour_colors=['blue', 'purple'], 
                legend_labels=['SNe', 'FRB + CC + SNe'])
g.export('Figuras/joint_mock.png', dpi=600)

In [ ]:
import pandas as pd

# Dicionário de amostras e nomes de arquivos
amostras = {
    "cosmography_frb_mock": samples_frb,
    # "cosmography_cc": samples_cc,
    # "cosmography_sne": samples_sne,
    "cosmography_joint_mock": samples_frb_sne_cc,
}

# Salva cada conjunto de amostras em arquivos CSV
for nome, dados in amostras.items():
    df = pd.DataFrame(dados)
    df.to_csv(f"Samplers/{nome}.csv", index=False)


In [ ]:
"""import pandas as pd
df1_frb = pd.read_csv("Samplers/samples1_frb.csv")
df1_sne = pd.read_csv("Samplers/samples1_sne.csv")
df1_sne_frb = pd.read_csv("Samplers/samples1_sne_frb.csv")
df2_frb = pd.read_csv("Samplers/samples2_frb.csv")
df2_sne = pd.read_csv("Samplers/samples2_sne.csv")
df2_sne_frb = pd.read_csv("Samplers/samples2_sne_frb.csv")"""

## Model Comparison

In [ ]:
"""samples = {
    "Sample_frb": {
        "results": {
            'wCDM': {'logz': wCDM_frb['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_frb['logz'], 'num_params': 6},
        }
    },
    "Sample_sne": {
        "results": {
            'wCDM': {'logz': wCDM_sne['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_sne['logz'], 'num_params': 6},
        }
    },
    "Sample_sne_frb": {
        "results": {
            'wCDM': {'logz': wCDM_sne_frb['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_sne_frb['logz'], 'num_params': 6},
        }
    }
}"""

In [ ]:
"""from bayesian_analysis import ModelComparison

comparison = ModelComparison(samples)
comparison.run_comparisons(save_to_file='comparisons_output_frb+SNe.txt')"""

In [ ]:
from bayesian_analysis import SaveResults

# Initialize the class with the desired output file
results_saver = SaveResults("Results_mcmc/results_cosmography_mock.txt")

# Optional: Clear the file if you want to start fresh
results_saver.reset_file()

results_saver.save_to_txt(mcsamples_frb, 'FRB_mock')
results_saver.save_to_txt(mcsamples_cc, 'CC')
results_saver.save_to_txt(mcsamples_sne, 'SNe')
results_saver.save_to_txt(mcsamples_frb_sne_cc, 'Joint_mock')

In [ ]:
from bayesian_analysis import CompareMCMCResults

comparator = CompareMCMCResults('Results_mcmc/mcmc_comparison_results_cosmography_mock.txt')

comparator.reset_file()

# Faz a comparação entre os dois modelos
comparator.compare_errors(mcsamples_frb, mcsamples_frb_sne_cc, "FRB_mock", "Joint_mock")
comparator.compare_errors(mcsamples_cc, mcsamples_frb_sne_cc, "CC", "Joint_mock")
comparator.compare_errors(mcsamples_sne, mcsamples_frb_sne_cc, "SNe", "Joint_mock")

In [ ]:
"""from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_16, y=dm_ext_obs_16, fmt='o', alpha=0.6, color='red', label='16 FRBs', ms=2)

z_test = np.linspace(0, 1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0, omega_a  in sampler_p2_16.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,  
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'))

band.line(color='k', linestyle='-', label='CPL parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_16.png', format='png', dpi=600)"""

In [ ]:
"""from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_63, y=dm_ext_obs_63, fmt='o', alpha=0.6, color='red', label='63 FRBs', ms=2)

z_test = np.linspace(0, 1.1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0  in sampler_constant_63.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant')
    )

band.line(color='k', linestyle='-', label='Constant parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_63.png', format='png', dpi=600)"""